<a href="https://colab.research.google.com/github/tanaymukherjee/Time-Series-Modeling/blob/master/9_The_GARCH_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Analysis

### Definition:
A sequence of information which attaches a time period to each value.

### Common Objective:
1. Determining the stability of financial markets and the efficiency portfolios.
2. Weather forcasting based on past records.

#### Import Libraries

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2 
from math import sqrt
import seaborn as sns
sns.set()

#### Mounting Google colab with personal GDrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#### Checking the content in my GDrive witihin the working directory

In [ ]:
!ls -ahl "/content/gdrive/My Drive/Colab Notebooks/Data"

#### Read the dataset

In [4]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Data/Index2018.csv")

#### View the dataset summary

Date column includes the timen period where as the other features tell us the stability of market.

1. SPX is US stock exchange by S&P 500
2. DAX is German stock exchange by  DAX 30
3. FTSE is UK stock exchange by FTSE 100
4. NIKKEI is Japan stock exchange by NIKKEI 225

#### Importing the Data and Pre-processing

In [6]:
raw_csv_data = data
df_comp=raw_csv_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst = True)
df_comp.set_index("date", inplace=True)
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [7]:
df_comp['market_value']=df_comp.ftse

In [8]:
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [9]:
import warnings
warnings.filterwarnings("ignore")

#### The LLR test

In [59]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p

### Creating Returns

In [10]:
df['returns'] = df.market_value.pct_change(1)*100

### The Simple GARCH Model